## 1. Import packages

In [1]:
# Import packages
import hail as hl
from bokeh.io import output_notebook,show
import gnomad.utils.vep
from hail.ggplot import *
import plotly
import plotly.io as pio
pio.renderers.default='iframe'

In [2]:
%run -i ~/BroadIS/utils/01_maps_funcs.py

## 2. Import data

In [3]:
# Import gnomaAD v.3.1.2
ht = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht')

# Import mutation rates from gnomAD paper
ht_mu = hl.import_table('data/supplementary_dataset_10_mutation_rates.tsv.gz',
                delimiter='\t', impute=True, force_bgz=True)
ht_mu = ht_mu.key_by('context', 'ref', 'alt', 'methylation_level') # has to have a key in order to join using foreign key

# Import context table from gnomad (https://broadinstitute.github.io/gnomad_methods/api_reference/utils/vep.html?highlight=context#gnomad.utils.vep.get_vep_context)
context_table = gnomad.utils.vep.get_vep_context("GRCh38").ht()
context_table = context_table.filter(hl.is_defined(context_table.methyl_mean))


context_table_parsed = context_table.select(context_table.context, context_table.methyl_mean)
context_table_parsed = context_table_parsed.transmute(context = context_table_parsed.context[2:5])

Initializing Hail with default parameters...


2022-10-04 16:50:55 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-10-04 16:50:57 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.124:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/01_maps/hail-20221004-1650-0.2.99-57537fea08d4.log
2022-10-04 16:51:10 Hail: INFO: Reading table to impute column types
2022-10-04 16:51:12 Hail: INFO: Finished type imputation
  Loading field 'context' as type str (imputed)
  Loading field 'ref' as type str (imputed)
  Loading field 'alt' as type str (imputed)
  Loading field 'methylation_level' as type int32 (imputed)
  Loading field 'mu_snp' as type float64 (imputed)


In [4]:
context_table_parsed = context_table_parsed.annotate(methyl_mean = hl.float64(context_table_parsed.methyl_mean))

In [5]:
context_table_parsed = context_table_parsed.annotate(methylation_level = hl.if_else(context_table_parsed.methyl_mean <= 0.2,0,
                                                            hl.if_else(context_table_parsed.methyl_mean <= 0.8,1,2)))

### Show the data structure

In [6]:
ht.show(3)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10031    | ["T","C"]  |
| chr1:10037    | ["T","C"]  |
| chr1:10043    | ["T","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [14149,7330,6840,3904,23217,2436,1647,594,176,118,49,29,19,9,11,6,5,2,4,2]   |
| [10121,5441,5610,3838,29394,3741,2868,1281,401,358,163,63,69,31,18,20,10,... |
| [6578,3782,4202,3336,31093,4532,3696,1752,595,585,261,88,82,59,17,26,14,5... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_q

In [7]:
# Table with methylation level and mutational rate in the trinucleotide context
ht_mu.show(3)

,,,,
context,ref,alt,methylation_level,mu_snp
str,str,str,int32,float64
"""AAA""","""A""","""C""",0,1.83e-09
"""AAA""","""A""","""G""",0,3.03e-09
"""AAA""","""A""","""T""",0,1.07e-09


In [8]:
# This table contains already precalculated nucleotides -3/+3 from mutation site 
context_table_parsed.show(3)

,,,,
locus,alleles,context,methyl_mean,methylation_level
locus<GRCh38>,array<str>,str,float64,int32
chr1:10469,"[""C"",""A""]","""TCG""",2.54e+01,2
chr1:10469,"[""C"",""G""]","""TCG""",2.54e+01,2
chr1:10469,"[""C"",""T""]","""TCG""",2.54e+01,2


## 3. Add context field to main data

In [9]:
# Before joining the tri-nucleotide context of mutation
ht.count() # data too big

759302267

In [10]:
# Join only matching rows from context to ht table.
#ht = ht.key_by('locus', 'alleles').join(context_table_parsed.key_by('locus', 'alleles'), how = 'left') # resorts the data making it slow
ht = ht.annotate(**context_table_parsed[ht.locus, ht.alleles])


In [11]:
# After
ht.count() # data too big

759302267

In [12]:
ht = ht.filter(hl.is_defined(ht.methylation_level))
ht = ht.head(10000) # Subset the data

ht.show(3)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10469    | ["C","A"]  |
| chr1:10469    | ["C","G"]  |
| chr1:10469    | ["C","T"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,6344,0),(25,1.61e-03,15518,0),(0,0.00e+00,3894,0),(0,0.00e+0... |
| [(60,9.71e-03,6182,0),(516,3.33e-02,15518,14),(39,1.04e-02,3766,0),(3,6.6... |
| [(0,0.00e+00,6348,0),(3,1.93e-04,15518,0),(0,0.00e+00,3898,0),(0,0.00e+00... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+---------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                       |
+---------------------------------------------------------------------------+
| array<int64>                                                              |
+---------------------------------------------------------------------------+
| [1070,934,915,849,1606,774,567,307,153,145,80,69,68,57,37,26,11,13,11,67] |
| [1070,934,915,849,1606,774,567,307,153,145,80,69,68,57,37,26,11,13,11,67] |
| [1070,934,915,849,1606,774,567,307,153,145,80,69,68,57,37,26,11,13,11,67] |
+---------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_freq         

## 4. Add mutation rates for added contexts

In [13]:
# Split alleles field to ref and alt allele
ht = ht.annotate(ref=ht.alleles[0], alt=ht.alleles[1])

# Add mutation rates according to the context, but also ref and alt allele for this context
#ht = ht.key_by("context", "ref", "alt").join(ht_mu.key_by("context", "ref", "alt"), how = 'left') # resorts the data making it slow
ht = ht.annotate(**ht_mu[ht.context, ht.ref, ht.alt, ht.methylation_level])


In [14]:
# After adding context and mutation rates to the main table 
# (can be more than original number of rows as context may occure more than once depending on the locus)
ht.count()

10000

In [15]:
# Show that contexts may be the same, but locus is completely different
ht.show(3)

2022-10-04 16:58:07 Hail: INFO: Ordering unsorted dataset with network shuffle9]
2022-10-04 16:58:08 Hail: INFO: Coerced sorted dataset
2022-10-04 16:58:29 Hail: INFO: Ordering unsorted dataset with network shuffle8]


+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10469    | ["C","A"]  |
| chr1:10469    | ["C","G"]  |
| chr1:10469    | ["C","T"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,6344,0),(25,1.61e-03,15518,0),(0,0.00e+00,3894,0),(0,0.00e+0... |
| [(60,9.71e-03,6182,0),(516,3.33e-02,15518,14),(39,1.04e-02,3766,0),(3,6.6... |
| [(0,0.00e+00,6348,0),(3,1.93e-04,15518,0),(0,0.00e+00,3898,0),(0,0.00e+00... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+---------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                       |
+---------------------------------------------------------------------------+
| array<int64>                                                              |
+---------------------------------------------------------------------------+
| [1070,934,915,849,1606,774,567,307,153,145,80,69,68,57,37,26,11,13,11,67] |
| [1070,934,915,849,1606,774,567,307,153,145,80,69,68,57,37,26,11,13,11,67] |
| [1070,934,915,849,1606,774,567,307,153,145,80,69,68,57,37,26,11,13,11,67] |
+---------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_freq         

## 5. Train linear model on synonymous variants for mutational class correction

In [16]:
ht_syn_ps = train_on_synonymous(ht)

2022-10-04 16:58:36 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'filtering_model' -> 'filtering_model_1'
    'dbsnp_version' -> 'dbsnp_version_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'vep_version' -> 'vep_version_1'
    'freq_meta' -> 'freq_meta_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'freq_sample_count' -> 'freq_sample_count_1'
    'faf_meta' -> 'faf_meta_1'
    'age_distribution' -> 'age_distribution_1'


In [17]:
ht_syn_ps.show(3)

2022-10-04 17:00:52 Hail: INFO: Ordering unsorted dataset with network shuffle9]
2022-10-04 17:00:52 Hail: INFO: Coerced sorted dataset
2022-10-04 17:01:12 Hail: INFO: Coerced sorted dataset             (9 + 9) / 18]
2022-10-04 17:03:28 Hail: INFO: Ordering unsorted dataset with network shuffle9]
2022-10-04 17:03:29 Hail: INFO: Coerced sorted dataset
2022-10-04 17:03:49 Hail: INFO: Coerced sorted dataset            (5 + 11) / 18]


,,,,,,,
context,ref,alt,mu_snp,methylation_level,N_variants,N_singletons,ps
str,str,str,float64,int32,int64,int64,float64
"""ACG""","""C""","""T""",1.42e-07,2,6,NA,NA
"""CCG""","""C""","""G""",NA,2,2,NA,NA
"""CCG""","""C""","""G""",NA,2,NA,1,NA


### Show input table for regression

In [ ]:
# How many rows after adding mutation rates
ht_syn_ps.count()

### Perform regression

In [ ]:
# Perform regression
ht_syn_lm = ht_syn_ps.aggregate(hl.agg.linreg(ht_syn_ps.ps, [1, ht_syn_ps.mu_snp], weight=ht_syn_ps.N_variants).beta)

# Show intercept and beta
ht_syn_lm

## 6. Predict expected number of variants for each context

### Function for regression eventually will be made starting here and put in `/utils/utils.py` script

In [ ]:
maps_table = regress_per_context(ht, ht_syn_lm)

In [ ]:
maps_table.show(maps_table.count())

In [ ]:
ggplot(maps_table, aes(x=maps_table.consequence, y = maps_table.maps)) + geom_col(aes(fill=maps_table.consequence))
